In [5]:
# import environment variables
from dotenv import load_dotenv
load_dotenv()

True

## agent 原理

![agent_workflow](./agent_workflow_diagram.png)

## agent 实现

### 定义模型

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = 'glm-4-flash',
    openai_api_base = "https://open.bigmodel.cn/api/paas/v4/",
)

### 定义 tools

#### 搜索 tool

In [7]:
from langchain_community.tools import DuckDuckGoSearchRun

#### 操作数据库 tool

In [8]:
import pandas as pd
from pandasql import sqldf

def simulate_database_operation(sql):
	my_table = pd.DataFrame({
	    'name': ['Henry Myers', 'Martha Hawkins', 'Kelsey Lutz', 'Jonathan Fowler', 'Jonathan Young', 'Autumn Johnson', 'Kimberly Macias', 'Jared Mccormick', 'Casey Hoover', 'Erica Morse'],
	    'age': [60, 44, 54, 46, 76, 22, 69, 33, 23, 35],
	    'sex': ['F', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'F', 'M']
	})
	result = sqldf(sql)
	return result

In [9]:
print(simulate_database_operation('SELECT * FROM my_table WHERE age > 50'))

              name  age sex
0      Henry Myers   60   F
1      Kelsey Lutz   54   M
2   Jonathan Young   76   F
3  Kimberly Macias   69   M


In [10]:
print(simulate_database_operation('SELECT * FROM my_table WHERE id > 50'))

PandaSQLException: (sqlite3.OperationalError) no such column: id
[SQL: SELECT * FROM my_table WHERE id > 50]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [11]:
from langchain.tools import tool


@tool
def simulate_database_operation(sql: str):
	'''根据sql语句操作数据库'''
	my_table = pd.DataFrame({
	    'name': ['Henry Myers', 'Martha Hawkins', 'Kelsey Lutz', 'Jonathan Fowler', 'Jonathan Young', 'Autumn Johnson', 'Kimberly Macias', 'Jared Mccormick', 'Casey Hoover', 'Erica Morse'],
	    'age': [60, 44, 54, 46, 76, 22, 69, 33, 23, 35],
	    'sex': ['F', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'F', 'M']
	})
	result = sqldf(sql)
	return result

### 模型调用 tool

In [12]:
tools = [DuckDuckGoSearchRun(), simulate_database_operation]
model_with_tools = model.bind_tools(tools)

In [15]:
from pprint import pprint as pp

response = model_with_tools.invoke('印度的首都是哪里？')
pp(dict(response))

{'additional_kwargs': {},
 'content': '印度的首都是新德里（New Delhi）。新德里不仅是印度的首都，也是印度联邦政府的行政中心。',
 'example': False,
 'id': 'run-c18946a2-43b0-4f2e-aa7c-87f0ed229d8c-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'stop',
                       'logprobs': None,
                       'model_name': 'glm-4-flash',
                       'system_fingerprint': None,
                       'token_usage': {'completion_tokens': 24,
                                       'prompt_tokens': 275,
                                       'total_tokens': 299}},
 'tool_calls': [],
 'type': 'ai',
 'usage_metadata': {'input_tokens': 275,
                    'output_tokens': 24,
                    'total_tokens': 299}}


In [16]:
fetch_response = model_with_tools.invoke('现在是北京时间几点？查询之后告诉我')
pp(dict(fetch_response))

{'additional_kwargs': {'tool_calls': [{'function': {'arguments': '{"query": '
                                                                 '"北京时间现在几点"}',
                                                    'name': 'duckduckgo_search'},
                                       'id': 'call_8994564354540219260',
                                       'index': 0,
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run-61357662-503a-4a2b-9773-dcd658b84c27-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'logprobs': None,
                       'model_name': 'glm-4-flash',
                       'system_fingerprint': None,
                       'token_usage': {'completion_tokens': 14,
                                       'prompt_tokens': 277,
                                       'total_tokens': 291}},
 'tool_calls': [{'args': {'query': '北京时间现在几点'},
           

In [19]:
db_response = model_with_tools.invoke('帮我往数据库的my_table表中插入一条数据，name是张三，age是18，sex是male')
pp(dict(db_response))

{'additional_kwargs': {'tool_calls': [{'function': {'arguments': '{"sql": '
                                                                 '"INSERT INTO '
                                                                 'my_table '
                                                                 '(name, age, '
                                                                 'sex) VALUES '
                                                                 "('张三', 18, "
                                                                 '\'male\')"}',
                                                    'name': 'simulate_database_operation'},
                                       'id': 'call_8994556657958401665',
                                       'index': 0,
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run-f32259cc-25a0-4655-9910-a91f2ff2ed32-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason':

### 定义agent

#### 手动agent

In [40]:
from icecream import ic

def manual_agent(query: str, model: ChatOpenAI, tools:list[tool]):
    model_with_tools = model.bind_tools(tools)
    model_output = model_with_tools.invoke(query)
    tool_response = call_tool(model_output, tools)
    final_response = model.invoke(
        f'original query: {query} \n\n\n tool response: {tool_response}',
    )
    return final_response


def call_tool(model_output, tools):
    tools_map = {tool.name.lower(): tool for tool in tools}
    tools_response = {}
    for tool in model_output.tool_calls:
        tool_name = tool['name']
        tool_args = tool['args']
        tool_instance = tools_map[tool_name]
        tool_response = tool_instance.invoke(*tool_args.values())
        tools_response[tool_name] = tool_response
    return tools_response

In [42]:
manual_agent('帮我查询数据库my_table表中有多少人年龄大于60', model, tools).content

'根据您提供的工具响应，查询结果显示数据库`my_table`表中年龄大于60岁的人数为2人。工具响应中的`COUNT(*)`表示统计了表中的所有记录数，而0和2分别表示查询的总记录数和符合条件的记录数。这里可能存在一个小的误解，因为`COUNT(*)`通常用于计算表中的总行数，而不是特定条件下的行数。正确的查询结果应该是：\n\n```\n查询结果显示数据库`my_table`表中年龄大于60岁的人数为2人。\n```'

#### 使用 langgraph 定义 agent

In [43]:
from langgraph.prebuilt import create_react_agent
agent = create_react_agent(model, tools)

In [45]:
from langchain_core.messages import HumanMessage
response = agent.invoke({'messages': [HumanMessage(content="今天在北京的天气怎么样")]})
response

{'messages': [HumanMessage(content='今天在北京的天气怎么样', id='635641b6-8328-4900-9016-9445acac4aa8'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8994560987285920653', 'function': {'arguments': '{"query": "北京今天天气"}', 'name': 'duckduckgo_search'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 275, 'total_tokens': 289}, 'model_name': 'glm-4-flash', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-43bde053-7a30-4431-91b7-763207b0afb8-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': '北京今天天气'}, 'id': 'call_8994560987285920653', 'type': 'tool_call'}], usage_metadata={'input_tokens': 275, 'output_tokens': 14, 'total_tokens': 289}),
  ToolMessage(content='北京天气预报，及时准确发布中央气象台天气信息，便捷查询北京今日天气，北京周末天气，北京一周天气预报，北京15日天气预报，北京天气预报还提供北京各区县的生活指数、健康指数、交通指数、旅游指数，及时发布北京气象预警信号、各类气象资讯。 北京市, 中国 - 当前温度和天气状况。今天详细的每小时天气预报 - 包括天气状况、温度、压力、湿度、降水、露点、风、能见度和紫外线指数数据。 中国天气网讯 今天（9月6日），北京天